In [47]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()
file_path = os.getenv("FILE_PATH")
file_path

'/Users/benjaminang/prac2025/stats-practitioner-challenge/challenge'

In [48]:
SNP_prices_file = f'{file_path}/S&P 500 Index.csv'
SNP_VIX_file = f'{file_path}/S&P 500 VIX.csv'

### S&P500 Prices

In [49]:
# Load the CSV file into df
SNP = pd.read_csv(SNP_prices_file)
SNP.columns = ["date", "price"]
SNP["date"] = pd.to_datetime(SNP["date"])
SNP = SNP.set_index("date")
SNP = SNP.sort_index(ascending=True)

# Calculating log prices
SNP["log_returns"] = np.log(SNP["price"]/ SNP["price"].shift(1))
SNP = SNP[1:]
SNP.head()

,price,log_returns
date,,
2015-01-05,2020.58,-0.018447
2015-01-06,2002.61,-0.008933
2015-01-07,2025.90,0.011563
2015-01-08,2062.14,0.017730
2015-01-09,2044.81,-0.008439


In [50]:
# import plotly.express as px
# import plotly.graph_objects as go

# # Assuming SNP is your DataFrame with 'date' and 'log_returns' columns
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=SNP["date"], 
#     y=SNP["log_returns"], 
#     mode='lines+markers', 
#     name="S&P 500 Index"
# ))

# fig.update_layout(
#     title="S&P 500 Index Time Series",
#     xaxis_title="Date",
#     yaxis_title="Index Value",
#     xaxis=dict(rangeslider=dict(visible=True)),  # Adds a range slider for better navigation
#     template="plotly_white"  # Clean background
# )

# fig.show()

In [51]:
# Define start and end dates
SNP_training_start, SNP_training_end = "2015-01-02", "2022-12-31"
SNP_testing_start, SNP_testing_end = "2023-01-03", "2023-02-01"

# Filter the DataFrame
SNP_training_df = SNP[(SNP.index >= SNP_training_start) & (SNP.index <= SNP_training_end)]
SNP_testing_df = SNP[(SNP.index >= SNP_testing_start) & (SNP.index <= SNP_testing_end)]
SNP_training_df.head()

,price,log_returns
date,,
2015-01-05,2020.58,-0.018447
2015-01-06,2002.61,-0.008933
2015-01-07,2025.90,0.011563
2015-01-08,2062.14,0.017730
2015-01-09,2044.81,-0.008439


In [52]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss

def stationarity_tests(series):
    """
    Performs ADF and KPSS tests on a time series.

    Parameters:
        series (pd.Series): The time series data.

    Returns:
        None (Prints test results)
    """
    # Augmented Dickey-Fuller Test (ADF)
    adf_result = adfuller(series, autolag='AIC')
    print("Augmented Dickey-Fuller Test Results:")
    print(f"ADF Statistic: {adf_result[0]:.4f}")
    print(f"p-value: {adf_result[1]:.4f}")
    print("Critical Values:")
    for key, value in adf_result[4].items():
        print(f"   {key}: {value:.4f}")
    print("➡ Stationary" if adf_result[1] < 0.05 else "➡ Non-Stationary")

    print("\n" + "="*50 + "\n")

    # KPSS Test
    kpss_result = kpss(series, regression='c', nlags="auto")  # 'c' = constant (level stationarity)
    print("KPSS Test Results:")
    print(f"KPSS Statistic: {kpss_result[0]:.4f}")
    print(f"p-value: {kpss_result[1]:.4f}")
    print("Critical Values:")
    for key, value in kpss_result[3].items():
        print(f"   {key}: {value:.4f}")
    print("➡ Non-Stationary" if kpss_result[1] < 0.05 else "➡ Stationary")

# Run the tests on S&P 500 log returns
stationarity_tests(SNP["log_returns"])

Augmented Dickey-Fuller Test Results:
ADF Statistic: -15.7254
p-value: 0.0000
Critical Values:
   1%: -3.4330
   5%: -2.8627
   10%: -2.5674
➡ Stationary


KPSS Test Results:
KPSS Statistic: 0.0495
p-value: 0.1000
Critical Values:
   10%: 0.3470
   5%: 0.4630
   2.5%: 0.5740
   1%: 0.7390
➡ Stationary


/Users/benjaminang/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/stattools.py:2022: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.




In [53]:
from hurst import compute_Hc

SNP_hurst_est, c, data = compute_Hc(SNP_training_df["log_returns"], kind='change', simplified=True)
print(f"Estimated Hurst Exponent: {SNP_hurst_est:.4f}")

Estimated Hurst Exponent: 0.5614


In [54]:
from stochastic.processes.continuous import FractionalBrownianMotion
import numpy as np
import random
import plotly.graph_objects as go

def simulate_fbm_prices(train_data, test_data, H):
    """
    Simulate future stock prices using a geometric fractional Brownian motion model.
    
    Parameters:
        train_data (DataFrame): Training data containing columns 'price' and 'log_returns'.
        test_data (DataFrame): Testing data (used to determine the forecast horizon).
        H (float): Hurst exponent for the fBM model.
    
    Returns:
        np.ndarray: Array of simulated future prices.
    """
    # Set a fixed seed for reproducibility
    random.seed(42)
    np.random.seed(42)
    
    # Calculate drift (μ) and volatility (σ) from training log returns
    mu = train_data["log_returns"].mean()
    sigma = train_data["log_returns"].std()

    # Define Fractional Brownian Motion (fBM) model
    n_days = len(test_data)
    fbm = FractionalBrownianMotion(hurst=H, t=n_days - 1)
    fbm_values = fbm.sample(n=n_days - 1)
    
    # Use the last known price from training data as the starting point
    S0 = train_data['price'].iloc[-1]
    simulated_prices = S0 * np.exp(mu * np.arange(n_days) + sigma * fbm_values)
    
    return simulated_prices

def plot_actual_vs_simulated(test_data, simulated_prices, H):
    """
    Create an interactive Plotly plot comparing the actual prices with simulated prices.
    
    Parameters:
        test_data (DataFrame): DataFrame containing the actual price data with its index as dates.
        simulated_prices (np.ndarray): Array of simulated prices.
        H (float): Hurst exponent used in the simulation (for plot title).
    """
    # Create a Plotly figure
    fig = go.Figure()

    # Add actual price trace
    fig.add_trace(go.Scatter(
        x=test_data.index,
        y=test_data["price"],
        mode='lines',
        name='Actual Price',
        line=dict(color='blue')
    ))

    # Add simulated (forecasted) price trace
    fig.add_trace(go.Scatter(
        x=test_data.index,
        y=simulated_prices,
        mode='lines',
        name='Forecasted Price (fBM)',
        line=dict(color='red', dash='dash')
    ))

    # Update layout of the plot
    fig.update_layout(
        title=f"Stock Price Forecast using fBM (H={H})",
        xaxis_title="Date",
        yaxis_title="Stock Price",
        template="plotly_white"
    )
    
    # Display the plot
    fig.show()

# Example usage:
simulated_prices = simulate_fbm_prices(SNP_training_df, SNP_testing_df, SNP_hurst_est)
plot_actual_vs_simulated(SNP_testing_df, simulated_prices, SNP_hurst_est)

In [55]:
def montecarlo_simulation_and_plot(train_data, test_data, H, num_simulations=100):
    """
    Perform a Monte Carlo simulation by generating multiple fBM price paths and plot them 
    alongside the actual stock prices.
    
    Parameters:
        train_data (DataFrame): Training data containing columns 'price' and 'log_returns'.
        test_data (DataFrame): Testing data (used to determine the forecast horizon).
        H (float): Hurst exponent for the fBM model.
        num_simulations (int): Number of simulation paths to generate.
    """
    # Set seeds for reproducibility
    random.seed(42)
    np.random.seed(42)
    
    # Calculate drift (μ) and volatility (σ) from training log returns
    mu = train_data["log_returns"].mean()
    sigma = train_data["log_returns"].std()
    
    n_days = len(test_data)
    time_points = np.arange(n_days)
    S0 = train_data['price'].iloc[-1]
    
    simulation_paths = []
    
    for i in range(num_simulations):
        # For each simulation, sample a new fBM path
        fbm = FractionalBrownianMotion(hurst=H, t=n_days - 1)
        fbm_values = fbm.sample(n=n_days - 2)
        fbm_values = np.insert(fbm_values, 0, 0)  # align with the time vector
        sim_prices = S0 * np.exp(mu * time_points + sigma * fbm_values)
        simulation_paths.append(sim_prices)
    
    # Create a Plotly figure for the Monte Carlo simulation
    fig = go.Figure()
    
    # Add each simulated path (using low opacity to avoid clutter)
    for i, sim_prices in enumerate(simulation_paths):
        fig.add_trace(go.Scatter(
            x=test_data.index,
            y=sim_prices,
            mode='lines',
            name=f'Simulation {i+1}',
            line=dict(color='rgba(255, 0, 0, 0.1)'),  # Red with transparency
            showlegend=False  # Hide individual simulation legends
        ))
    
    # Overlay the actual prices
    fig.add_trace(go.Scatter(
        x=test_data.index,
        y=test_data["price"],
        mode='lines',
        name='Actual Price',
        line=dict(color='blue', width=2)
    ))
    
    # Update plot layout
    fig.update_layout(
        title=f"Monte Carlo Simulation of Stock Prices using fBM (H={H}) - {num_simulations} Paths",
        xaxis_title="Date",
        yaxis_title="Stock Price",
        template="plotly_white"
    )
    
    fig.show()

montecarlo_simulation_and_plot(SNP_training_df, SNP_testing_df, SNP_hurst_est, num_simulations=100)

In [ ]:
# Compute rolling Hurst

def compute_rolling_hurst(df, window, col='log_returns', lag=0):
    hurst_values = []
    times = []
    
    # Loop through the DataFrame using the rolling window
    for i in range(window - 1, len(df), lag+1):
        # Extract the window slice from the series
        window_series = df[col].iloc[i - window + 1 : i + 1: lag+1]
        # Compute the Hurst exponent using the 'change' method and simplified calculation
        h, c, data = compute_Hc(window_series, kind='change', simplified=True)
        
        hurst_values.append(h)
        times.append(df.index[i])
    
    # Create and return a new DataFrame with the computed Hurst exponents
    result_df = pd.DataFrame({'hurst': hurst_values}, index=times)
    return result_df

hurst_window150_lag0 = compute_rolling_hurst(SNP, window=150, col='log_returns', lag=0)
hurst_window350_lag0 = compute_rolling_hurst(SNP, window=350, col='log_returns', lag=0)
hurst_window500_lag0 = compute_rolling_hurst(SNP, window=500, col='log_returns', lag=0)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=hurst_window150_lag0.index,
    y=hurst_window150_lag0["hurst"],
    mode='lines',
    name='150 days rolling window',
    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=hurst_window350_lag0.index,
    y=hurst_window350_lag0["hurst"],
    mode='lines',
    name='350 days rolling window',
    line=dict(color='red')
))
fig.add_trace(go.Scatter(
    x=hurst_window500_lag0.index,
    y=hurst_window500_lag0["hurst"],
    mode='lines',
    name='500 days rolling window',
    line=dict(color='green')
))
fig.update_layout(
    title=f"Hurst exponent for SNP",
    xaxis_title="Date",
    yaxis_title="Hurst Value",
    template="plotly_white"
)

In [57]:
def compute_hit_ratio(df, hurst_col='hurst', return_col='log_returns', threshold=0.5):
    """
    Compute the hit ratio for a trading strategy based on the Hurst exponent.
    
    Parameters:
    df : DataFrame
        DataFrame containing Hurst exponent and log returns.
    hurst_col : str
        Column name for the Hurst exponent.
    return_col : str
        Column name for the log returns.
    threshold : float
        Threshold to determine trend-following (> threshold) or mean-reverting (< threshold).
    
    Returns:
    float
        Hit ratio (proportion of correct predictions).
    """
    df = df.dropna().copy()  # Ensure no NaNs

    # Generate trading signals based on Hurst exponent
    df['signal'] = np.where(df[hurst_col] > threshold, 1, -1)  # 1 for trending, -1 for mean-reverting

    # Shift returns forward to compare with the signal
    df['future_return'] = df[return_col].shift(-1)  # Next-period return

    # Correct predictions: signal * future return > 0 means correct prediction
    df['correct'] = np.sign(df['signal'] * df['future_return']) == 1

    # Compute hit ratio
    hit_ratio = df['correct'].mean()
    
    return hit_ratio

hit_150_0_df = SNP.join(hurst_window150_lag0, how='inner')
hit_350_0_df = SNP.join(hurst_window350_lag0, how='inner')
hit_500_0_df = SNP.join(hurst_window500_lag0, how='inner')

hit_150_0 = compute_hit_ratio(hit_150_0_df)
hit_350_0 = compute_hit_ratio(hit_350_0_df)
hit_500_0 = compute_hit_ratio(hit_500_0_df)

print(f"Hit Ratios:\n"
      f"- 150-day window (0 lag): {hit_150_0:.2%}\n"
      f"- 350-day window (0 lag): {hit_350_0:.2%}\n"
      f"- 500-day window (0 lag): {hit_500_0:.2%}")

Hit Ratios:
- 150-day window (0 lag): 51.86%
- 350-day window (0 lag): 51.06%
- 500-day window (0 lag): 49.45%


In [ ]:
# Using different lags
hurst_window150_lag2 = compute_rolling_hurst(SNP, window=150, col='log_returns', lag=0)
hurst_window350_lag2 = compute_rolling_hurst(SNP, window=350, col='log_returns', lag=2)
hurst_window500_lag2 = compute_rolling_hurst(SNP, window=500, col='log_returns', lag=4)

hit_150_2_df = SNP.join(hurst_window150_lag2, how='inner')
hit_350_2_df = SNP.join(hurst_window350_lag2, how='inner')
hit_500_2_df = SNP.join(hurst_window500_lag2, how='inner')

hit_150_2 = compute_hit_ratio(hit_150_2_df)
hit_350_2 = compute_hit_ratio(hit_350_2_df)
hit_500_2 = compute_hit_ratio(hit_500_2_df)

print(f"Hit Ratios:\n"
      f"- 150-day window: {hit_150_2:.2%}\n"
      f"- 350-day window: {hit_350_2:.2%}\n"
      f"- 500-day window: {hit_500_2:.2%}")

Hit Ratios:
- 150-day window: 51.86%
- 350-day window: 51.25%
- 500-day window: 49.75%


In [ ]:
def plot_actual_vs_predicted_plotly(df, hurst_col='hurst', price_col='price', return_col='log_returns', threshold=0.5):
    """
    Plot actual vs predicted prices interactively using Plotly, based on the Hurst exponent strategy.
    
    Parameters:
    df : DataFrame
        Data containing price, log returns, and Hurst exponent.
    hurst_col : str
        Column name for the Hurst exponent.
    price_col : str
        Column name for the actual price.
    return_col : str
        Column name for log returns.
    threshold : float
        Threshold to determine trend-following (> threshold) or mean-reverting (< threshold).
    """
    df = df.dropna().copy()  # Drop NaN values

    # Generate trading signals
    df['signal'] = np.where(df[hurst_col] > threshold, 1, -1)

    # Predict the next price using the signal and log returns
    df['predicted_price'] = df[price_col] * np.exp(df['signal'] * df[return_col].shift(-1))

    # Create the interactive plot
    fig = go.Figure()

    # Add actual price trace
    fig.add_trace(go.Scatter(x=df.index, y=df[price_col], mode='lines', name='Actual Price', line=dict(color='blue')))

    # Add predicted price trace
    fig.add_trace(go.Scatter(x=df.index, y=df['predicted_price'], mode='lines', name='Predicted Price', 
                             line=dict(color='red', dash='dot')))

    # Layout customization
    fig.update_layout(title="Actual vs Predicted Prices using Hurst Strategy",
                      xaxis_title="Date",
                      yaxis_title="Price",
                      legend=dict(x=0, y=1),
                      hovermode="x unified")

    fig.show()

# Example Usage:
plot_actual_vs_predicted_plotly(hit_150_df)  # Use hit_350_df or hit_500_df for different rolling windows